In [ ]:
!pip install transformers tensorflow numpy scikit-learn

In [ ]:
import pandas as pd
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

Caculate similarity score for Prompts

In [ ]:
import pandas as pd
from transformers import BertTokenizer, TFBertModel
from sklearn.metrics.pairwise import cosine_similarity

# Load the CSV file into a DataFrame
df = pd.read_csv('/content/drive/MyDrive/Method/EnglishMethod/Score-Examples/Prompt-Final.csv')
df.head()

,Functional Description,Method Name,average_score,MTP,COT
0,get the cursor of this object,getCursor,0.894796,getCursor,getCursor
1,returns a hash code value for this page attrib...,hashCode,0.894776,getPageAttributesHashCode,getPageAttributesHashCode
2,"deprecated as of jdk version 1 1 , replaced by...",mouseUp,0.894730,processMouseEvent,processMouseEvent
3,gets the method that this method descriptor en...,getMethod,0.894617,getEncapsulatedMethod,getEncapsulatedMethod
4,gets the state of this frame obsolete,getState,0.894594,getFrameState,getFrameState


In [ ]:
def convert_java_method_name_to_string(method_name):
    # Split the method name based on uppercase letters while keeping them
    words = []
    start_index = 0
    for i in range(1, len(method_name)):
        if method_name[i].isupper():
            words.append(method_name[start_index:i])
            start_index = i
    words.append(method_name[start_index:])

    # Join the words with spaces and capitalize the first letter of each word
    result_string = ' '.join(word for word in words)
    return result_string

In [ ]:
df['MN'] = df['Method Name'].apply(convert_java_method_name_to_string)
df['MTP1'] = df['MTP'].apply(convert_java_method_name_to_string)
df['COT1'] = df['COT'].apply(convert_java_method_name_to_string)
#df['Maieutic1'] = df['Maieutic'].apply(convert_java_method_name_to_string)
#df['GD1'] = df['GD'].apply(convert_java_method_name_to_string)
#df['Socratic1'] = df['Socratic'].apply(convert_java_method_name_to_string)
#df['SB1'] = df['SB'].apply(convert_java_method_name_to_string)
#df['Analogical1'] = df['Analogical'].apply(convert_java_method_name_to_string)
df.head()

,Functional Description,Method Name,average_score,MTP,COT,MN,MTP1,COT1
0,get the cursor of this object,getCursor,0.894796,getCursor,getCursor,get Cursor,get Cursor,get Cursor
1,returns a hash code value for this page attrib...,hashCode,0.894776,getPageAttributesHashCode,getPageAttributesHashCode,hash Code,get Page Attributes Hash Code,get Page Attributes Hash Code
2,"deprecated as of jdk version 1 1 , replaced by...",mouseUp,0.894730,processMouseEvent,processMouseEvent,mouse Up,process Mouse Event,process Mouse Event
3,gets the method that this method descriptor en...,getMethod,0.894617,getEncapsulatedMethod,getEncapsulatedMethod,get Method,get Encapsulated Method,get Encapsulated Method
4,gets the state of this frame obsolete,getState,0.894594,getFrameState,getFrameState,get State,get Frame State,get Frame State


In [ ]:
# Load the pre-trained BERT model and tokenizer
model_name = "bert-base-multilingual-cased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = TFBertModel.from_pretrained(model_name)

# List of column names to compare with "Method Name"
comparison_columns = ["MTP1",	"COT1"]

# Iterate through each row and calculate similarity scores for each comparison column
for col in comparison_columns:
    similarity_scores = []
    for index, row in df.iterrows():  # Iterating over all rows in the dataset
        # Tokenize and encode the sentences
        encoded_method_name = tokenizer(row["MN"], padding=True, truncation=True, return_tensors="tf")
        encoded_other = tokenizer(row[col], padding=True, truncation=True, return_tensors="tf")

        # Generate embeddings for the sentences
        embed_method_name = model(encoded_method_name).last_hidden_state[:, 0, :].numpy()
        embed_other = model(encoded_other).last_hidden_state[:, 0, :].numpy()

        # Calculate cosine similarity between embeddings
        similarity_score = cosine_similarity(embed_method_name, embed_other)[0][0]

        # Append the similarity score to the list
        similarity_scores.append(similarity_score)

    # Add similarity scores to the DataFrame
    df[f"similarity_with_{col}"] = similarity_scores

# Print the DataFrame with similarity scores


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [ ]:
df.head()

,Functional Description,Method Name,average_score,MTP,COT,MN,MTP1,COT1,similarity_with_MTP1,similarity_with_COT1
0,get the cursor of this object,getCursor,0.894796,getCursor,getCursor,get Cursor,get Cursor,get Cursor,1.000000,1.000000
1,returns a hash code value for this page attrib...,hashCode,0.894776,getPageAttributesHashCode,getPageAttributesHashCode,hash Code,get Page Attributes Hash Code,get Page Attributes Hash Code,0.706119,0.706119
2,"deprecated as of jdk version 1 1 , replaced by...",mouseUp,0.894730,processMouseEvent,processMouseEvent,mouse Up,process Mouse Event,process Mouse Event,0.789693,0.789693
3,gets the method that this method descriptor en...,getMethod,0.894617,getEncapsulatedMethod,getEncapsulatedMethod,get Method,get Encapsulated Method,get Encapsulated Method,0.910814,0.910814
4,gets the state of this frame obsolete,getState,0.894594,getFrameState,getFrameState,get State,get Frame State,get Frame State,0.919891,0.919891


In [ ]:
similarity_columns = [
    "similarity_with_MTP1", "similarity_with_COT1"
]

# Calculate and print average scores for each similarity column
for col in similarity_columns:
    average_score = df[col].mean()
    print(f"Average score of {col}: {average_score}")

Average score of similarity_with_MTP1: 0.8801881074905396
Average score of similarity_with_COT1: 0.8603798151016235
